# keys

> Cryptographic and key management helpers for interacting with the Sherlock Domains API

In [ ]:
#| default_exp keys

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

Clients authenticate using public key cryptography. Currently, identities are based on the ed25519 key pairs.

In [ ]:
#| export
from cryptography.hazmat.primitives.asymmetric import ed25519
from cryptography.hazmat.primitives import serialization


In [ ]:
priv_key = ed25519.Ed25519PrivateKey.generate()

priv_key_hex = priv_key.private_bytes(encoding=serialization.Encoding.Raw, format=serialization.PrivateFormat.Raw, encryption_algorithm=serialization.NoEncryption()).hex()
pub_key_hex = priv_key.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw).hex()

{
    "private_key_hex": priv_key_hex,
    "public_key_hex": pub_key_hex
}


{'private_key_hex': '3499a4516d7a41ee4ed9fc56e3afa9810928777ee25e35c694bd3575ed0f0d87',
 'public_key_hex': '1cead4f0b4289cf1b20fedb08b8fbfb7bf0e2f89875b59f1ce0761b182f8bfbb'}

In [ ]:
#| export
class Key:
    def __init__(self, 
                 priv_key: ed25519.Ed25519PrivateKey):
        self.priv_key = priv_key
        self.pub_key = priv_key.public_key()

    def public_key(self) -> str:
        return self.pub_key.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw).hex()
    
    def export(self) -> dict:
        return {
            "private_key_hex": self.priv_key.private_bytes(encoding=serialization.Encoding.Raw, format=serialization.PrivateFormat.Raw, encryption_algorithm=serialization.NoEncryption()).hex(),
            "public_key_hex": self.pub_key.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw).hex()
        }
    
    def sign(self, message: bytes) -> bytes:
        return self.priv_key.sign(message)
    
    def __str__(self):
        return f"Key(pub_key_hex={self.pub_key.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw).hex()})"

    def __repr__(self):
        return self.__str__()

In [ ]:
#| export
def generate_key() -> Key:
    """Generate a new key pair for a client"""
    return Key(ed25519.Ed25519PrivateKey.generate())


In [ ]:
key = generate_key()
key

Key(pub_key_hex=2aaf406e5dee9f6e732a5a466f96de48fd6747dba429a057608c7bf70e52a3d9)

In [ ]:
#| export
def load_key(priv_key_hex: str) -> Key:
    """Load a key pair from an ed25519 hex encoded private key"""
    return Key(ed25519.Ed25519PrivateKey.from_private_bytes(bytes.fromhex(priv_key_hex)))

In [ ]:
key = load_key("3557ed77dd346fcbeefff47115622aa449c0054384347aaa2a6d1284d54caf6d")
test_eq(key.export()['private_key_hex'], "3557ed77dd346fcbeefff47115622aa449c0054384347aaa2a6d1284d54caf6d")
test_eq(key.export()['public_key_hex'], "c695748b10043c10ce2af90a227ebe495a8e5a6096f5f9832088b963309517be")
test_eq(key.public_key(), "c695748b10043c10ce2af90a227ebe495a8e5a6096f5f9832088b963309517be")

In [ ]:
message = "hello"

msg_bytes = message.encode()
signed_msg = key.sign(msg_bytes)
encoded_signature = signed_msg.hex()

test_eq(encoded_signature,"7f7ec3dc5b61352adbdced1766d944aa3bd243b6b231ad08a5607373fb3bea0cca965d2924e6cfeb7440d2d8b52ba5f24b61954e457337253aca989563404c01")